# Analizador Lexico (Con más de una función)

### Analisis Léxico

In [416]:
import re
import json
import os
import subprocess
import platform

In [417]:
# === Analisis léxico ===
# Definir los patrones

token_patron = {
    "KEYWORD": r"\b(if|else|while|return|int|float|void|for|print)\b",
    "IDENTIFIER": r"\b[a-zA-Z_][a-zA-Z0-9_]*\b",
    "NUMBER": r"\b\d+(\.\d+)?\b",
    "OPERATOR": r"(==|!=|<=|>=|&&|\|\||\+\+|--|[+\-*/=<>!&])",
    "STRING": r'"([^"]*)"',
    "DELIMITER": r"[(),;{}]",
    "WHITESPACE": r"\s+",
}

def identificar(texto):
    # Unir todos los patrones en un único patrón utilizando grupos nombrados
    patron_general = "|".join(f"(?P<{token}>{patron})" for token, patron in token_patron.items())
    patron_regex = re.compile(patron_general)
    tokens_encontrados = []
    for match in patron_regex.finditer(texto):
        for token, valor in match.groupdict().items():
            if valor is not None and token != "WHITESPACE":
                tokens_encontrados.append((token, valor))
    return tokens_encontrados

### Parsear

In [418]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        
    def obtener_token_actual(self):
        return self.tokens[self.pos] if self.pos < len(self.tokens) else None

    def coincidir(self, tipo_esperado):
        token_actual = self.obtener_token_actual()
        if token_actual and token_actual[0] == tipo_esperado:
            self.pos += 1
            return token_actual
        else:
            raise SyntaxError(f'Error sintactico: se esperaba {tipo_esperado}, pero se encontro: {token_actual}')

    def parsear(self):
        # Ahora parsear retorna un nodo programa que contiene todas las funciones
        funciones = []
        
        while self.pos < len(self.tokens):
            funciones.append(self.funcion())
            
        programa = NodoPrograma(funciones)
        
        # Verificar que exista la función main
        if not programa.tiene_main():
            print("Advertencia: No se encontró la función main() en el programa")
            
        return programa
  
    def funcion(self):
        tipo_token = self.coincidir('KEYWORD')  # Tipo de retorno (int/float)
        tipo = tipo_token[1].lower()  # Convertir a minúsculas para consistencia
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la funcion
        nombre = nombre_token[1]
        
        self.coincidir('DELIMITER')  # Se espera un parentesis
        parametros = self.parametros()
        self.coincidir('DELIMITER')  # Se espera un parentesis de cierre ")"
        self.coincidir('DELIMITER')  # Se espera una llave de apertura "{"
        
        cuerpo = self.cuerpo()  # Se espera el cuerpo de la funcion
        
        self.coincidir('DELIMITER')  # Se espera una llave de cierre "}"
        
        return NodoFuncion(tipo, nombre, parametros, cuerpo)
    
    def parametros(self):
        parametros = []
        
        if self.obtener_token_actual() and self.obtener_token_actual()[1] != ')':
            tipo_token = self.coincidir('KEYWORD')  # Tipo del parametro
            tipo = tipo_token[1].lower()  # Convertir a minúsculas
            nombre_token = self.coincidir('IDENTIFIER')  # Nombre del parametro
            nombre = nombre_token[1]
            
            parametros.append(NodoParametro(tipo, nombre))
            
            while self.obtener_token_actual() and self.obtener_token_actual()[1] == ',':
                self.coincidir('DELIMITER')  # Se espera una coma ","
                tipo_token = self.coincidir('KEYWORD')  # Tipo del parametro
                tipo = tipo_token[1].lower()
                nombre_token = self.coincidir('IDENTIFIER')  # Nombre del parametro
                nombre = nombre_token[1]
                
                parametros.append(NodoParametro(tipo, nombre))
            
        return parametros

    def cuerpo(self):
        instrucciones = []
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] != '}':
            if self.obtener_token_actual()[1] == 'return':
                instrucciones.append(self.declaracion())
            elif self.obtener_token_actual()[1] == 'while':
                instrucciones.append(self.ciclo_while())
            elif self.obtener_token_actual()[1] == 'if':
                instrucciones.append(self.condicional_if())
            elif self.obtener_token_actual()[1] == 'else':
                instrucciones.append(self.condicional_else())
            elif self.obtener_token_actual()[1] == 'for':
                instrucciones.append(self.ciclo_for())
            elif self.obtener_token_actual()[1] == 'print':
                instrucciones.append(self.imprimir())
            elif self.obtener_token_actual()[0] == 'IDENTIFIER':
                # Verificar si es una llamada a función o un incremento
                siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
                if siguiente and siguiente[1] == '(':
                    instrucciones.append(self.llamada_funcion())
                    self.coincidir('DELIMITER')  # Punto y coma ";"
                elif siguiente and siguiente[0] == 'OPERATOR' and siguiente[1] in ('++', '--'):
                    instruccion = self.incremento()
                    instrucciones.append(instruccion)
                    self.coincidir('DELIMITER')  # Punto y coma ";"
                else:
                    # Es una asignación a una variable existente
                    nombre_token = self.coincidir('IDENTIFIER')
                    nombre = nombre_token[1]
                    self.coincidir('OPERATOR')  # =
                    expresion = self.expresion()
                    self.coincidir('DELIMITER')  # ;
                    instrucciones.append(NodoAsignacion(None, nombre, expresion))
            elif self.obtener_token_actual()[0] == 'KEYWORD':
                # Es una declaración de variable con posible asignación
                instrucciones.append(self.asignacion())
            else:
                raise SyntaxError(f"Error sintáctico: instrucción inesperada {self.obtener_token_actual()}")
                
        return instrucciones

    def llamada_funcion(self):
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la función
        nombre = nombre_token[1]
        
        self.coincidir('DELIMITER')  # (
        
        argumentos = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] != ')':
            argumentos.append(self.expresion())
            
            while self.obtener_token_actual() and self.obtener_token_actual()[1] == ',':
                self.coincidir('DELIMITER')  # ,
                argumentos.append(self.expresion())
                
        self.coincidir('DELIMITER')  # )
        
        return NodoLlamadaFuncion(nombre, argumentos)

    def asignacion(self):
        tipo_token = self.coincidir('KEYWORD')  # Tipo de variable (ej. int)
        tipo = tipo_token[1]
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la variable
        nombre = nombre_token[1]
        self.coincidir('OPERATOR')  # Operador de asignación "="
        expresion = self.expresion()  
        self.coincidir('DELIMITER')  # Punto y coma ";"
        
        return NodoAsignacion(tipo, nombre, expresion)

    def declaracion(self):
        self.coincidir('KEYWORD')  # return
        expresion = self.expresion()
        self.coincidir('DELIMITER')  # Punto y coma ";"
        
        return NodoRetorno(expresion)

    def expresion(self):
        izquierda = self.termino()
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('+', '-', '==', '!=', '<', '>', '<=', '>=', '&&', '||'):
            operador_token = self.coincidir('OPERATOR')
            operador = operador_token[1]
            derecha = self.termino()
            izquierda = NodoOperacion(izquierda, operador, derecha)
            
        return izquierda

    def termino(self):
        izquierda = self.factor()
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('*', '/'):
            operador_token = self.coincidir('OPERATOR')
            operador = operador_token[1]
            derecha = self.factor()
            izquierda = NodoOperacion(izquierda, operador, derecha)
            
        return izquierda

    def factor(self):
        token_actual = self.obtener_token_actual()
        
        if token_actual[0] == 'NUMBER':
            token = self.coincidir('NUMBER')
            return NodoNumero(token[1])
        elif token_actual[0] == 'IDENTIFIER':
            # Verificar si es una llamada a función o una variable
            siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if siguiente and siguiente[1] == '(':
                return self.llamada_funcion()
            else:
                token = self.coincidir('IDENTIFIER')
                return NodoIdentificador(token[1])
        elif token_actual[1] == '(':
            self.coincidir('DELIMITER')  # "("
            expresion = self.expresion()
            self.coincidir('DELIMITER')  # ")"
            return expresion
        else:
            raise SyntaxError(f'Error sintáctico en factor: token inesperado {token_actual}')

    def incremento(self):
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la variable
        nombre = nombre_token[1]
        
        operador_token = self.coincidir('OPERATOR')  # Coincide con ++ o --
        operador = operador_token[1]
        
        return NodoIncremento(nombre, operador)
      
    def instruccion_unica(self):
        token_actual = self.obtener_token_actual()
        if not token_actual:
            raise SyntaxError("Error sintáctico: se esperaba una instrucción")

        if token_actual[0] == "KEYWORD":
            if token_actual[1] == "return":
                return self.declaracion()
            elif token_actual[1] == "while":
                return self.ciclo_while()
            elif token_actual[1] == "if":
                return self.condicional_if()
            elif token_actual[1] == "else":
                return self.condicional_else()
            elif token_actual[1] == "for":
                return self.ciclo_for()
            elif token_actual[1] == "print":
                return self.imprimir()
            else:
                return self.asignacion()
        elif token_actual[0] == "IDENTIFIER":
            # Verificar si es una llamada a función
            siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if siguiente and siguiente[1] == '(':
                instruccion = self.llamada_funcion()
                self.coincidir('DELIMITER')  # ;
                return instruccion
            else:
                return self.asignacion()
        else:
            raise SyntaxError(f"Error sintáctico: instrucción inesperada {token_actual}")

    def ciclo_while(self):
        self.coincidir('KEYWORD')  # while
        self.coincidir('DELIMITER')  # "("
        condicion = self.expresion()  # Condición del ciclo
        self.coincidir('DELIMITER')  # ")"

        # Verificar si el siguiente token es una llave de apertura "{"
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del while
            self.coincidir('DELIMITER')  # "}"
        else:
            # Si no hay llaves, solo ejecutamos una única instrucción
            cuerpo.append(self.instruccion_unica())
            
        return NodoWhile(condicion, cuerpo)

    def ciclo_for(self):
        self.coincidir('KEYWORD')  # for
        self.coincidir('DELIMITER')  # "("
        
        # Inicialización
        if self.obtener_token_actual()[0] == 'KEYWORD':
            inicializacion = self.asignacion()
        else:
            nombre_token = self.coincidir('IDENTIFIER')
            nombre = nombre_token[1]
            self.coincidir('OPERATOR')  # =
            expresion = self.expresion()
            self.coincidir('DELIMITER')  # ";"
            inicializacion = NodoAsignacion(None, nombre, expresion)
            
        # Condición
        condicion = self.expresion()
        self.coincidir('DELIMITER')  # ";"
        
        # Incremento
        incremento = self.incremento()
        self.coincidir('DELIMITER')  # ")"
        
        # Cuerpo
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del for
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        return NodoFor(inicializacion, condicion, incremento, cuerpo)

    def condicional_if(self):
        self.coincidir('KEYWORD')  # if
        self.coincidir('DELIMITER')  # "("
        condicion = self.expresion()  # Condición del if
        self.coincidir('DELIMITER')  # ")"

        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del if
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        # Verificar si hay else
        cuerpo_else = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "else":
            self.coincidir('KEYWORD')  # else
            if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
                self.coincidir('DELIMITER')  # "{"
                cuerpo_else = self.cuerpo()  # Cuerpo del else
                self.coincidir('DELIMITER')  # "}"
            else:
                cuerpo_else.append(self.instruccion_unica())
            
        return NodoIf(condicion, cuerpo, cuerpo_else)

    def condicional_else(self):
        self.coincidir('KEYWORD')  # else
        
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del else
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        return NodoElse(cuerpo)

    def imprimir(self):
        self.coincidir('KEYWORD')  # print
        self.coincidir('DELIMITER')  # "("
        
        token_actual = self.obtener_token_actual()
        if token_actual[0] == 'STRING':
            token = self.coincidir('STRING')
            expresion = NodoString(token[1])
        else:
            expresion = self.expresion()  # Expresion a imprimir
    
        self.coincidir('DELIMITER')  # ")"
        self.coincidir('DELIMITER')  # ";"
        
        return NodoPrint(expresion)

In [419]:
class NodoAST:
    # Clase base para todos los nodos del AST
    def to_dict(self):
        # Método para convertir nodo a diccionario (para JSON)
        raise NotImplementedError("Debe implementarse en las subclases")
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        raise NotImplementedError("Metodo traducir() no implementado en este nodo")

In [420]:
class NodoPrograma(NodoAST):
    def __init__(self, funciones):
        self.funciones = funciones
        
    def to_dict(self):
        return {
            "tipo": "programa",
            "funciones": [funcion.to_dict() for funcion in self.funciones]
        }
    
    def tiene_main(self):
        for funcion in self.funciones:
            if funcion.nombre == "main":
                return True
        return False

In [421]:
class NodoFuncion(NodoAST):
    def __init__(self, tipo, nombre, parametros, cuerpo):
        self.tipo = tipo
        self.nombre = nombre
        self.parametros = parametros
        self.cuerpo = cuerpo
    
    def traducir(self):
        params = ".".join(p.traducir() for p in self.parametros)
        cuerpo = "\n    ".join(c.traducir() for c in self.cuerpo)
        return f"def {self.nombre}({params}):\n    {cuerpo}"
        

    def to_dict(self):
        return {
            "tipo": "funcion",
            "retorno": self.tipo,
            "nombre": self.nombre,
            "parametros": [param.to_dict() for param in self.parametros],
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [422]:
class NodoParametro(NodoAST):
    def __init__(self, tipo, nombre):
        self.tipo = tipo
        self.nombre = nombre

    def traducir(self):
        return self.nombre
    
    def to_dict(self):
        return {
            "tipo": "parametro",
            "tipo_dato": self.tipo,
            "nombre": self.nombre
        }

In [423]:
class NodoAsignacion(NodoAST):
    def __init__(self, tipo, nombre, expresion):
        self.tipo = tipo
        self.nombre = nombre
        self.expresion = expresion
        
    def traducir(self):
        return f"{self.nombre} = {self.expresion.traducir()}"
        
    def to_dict(self):
        return {
            "tipo": "asignacion",
            "tipo_dato": self.tipo,
            "nombre": self.nombre,
            "expresion": self.expresion.to_dict()
        }

In [424]:
class NodoOperacion(NodoAST):
    def __init__(self, izquierda, operador, derecha):
        self.izquierda = izquierda
        self.operador = operador
        self.derecha = derecha
             
    def to_dict(self):
        return {
            "tipo": "operacion",
            "operador": self.operador,
            "izquierda": self.izquierda.to_dict(),
            "derecha": self.derecha.to_dict()
        }
        
    def traducir(self):
        if self.operador == '+':
            return f"({self.izquierda.traducir()} + {self.derecha.traducir()})"
        elif self.operador == '-':
            return f"({self.izquierda.traducir()} - {self.derecha.traducir()})"
        elif self.operador == '*':
            return f"({self.izquierda.traducir()} * {self.derecha.traducir()})"
        elif self.operador == '/':
            return f"({self.izquierda.traducir()} / {self.derecha.traducir()})"
        
    def optimizacion(self):
        izquierda = self.izquierda.optimizacion()
        derecha = self.derecha.optimizacion()
        
        # Si ambos operandos son numeros, evaluamos la operacion
        if isinstance(izquierda, self.NodoNumero) and isinstance(derecha, self.NodoNumero):
            if self.operador == '+':
                return self.NodoNumero(float(izquierda.valor) + float(derecha.valor))
            elif self.operador == '-':
                return self.NodoNumero(float(izquierda.valor) - float(derecha.valor))
            elif self.operador == '*':
                return self.NodoNumero(float(izquierda.valor) * float(derecha.valor))  # Corregido: * en lugar de +
            elif self.operador == '/' and float(derecha.valor) != 0:
                return self.NodoNumero(float(izquierda.valor) / float(derecha.valor))
            
        # Simplificacion algebraica
        if self.operador == '*' and isinstance(derecha, self.NodoNumero) and float(derecha.valor) == 1:
            return izquierda
        if self.operador == '*' and isinstance(izquierda, self.NodoNumero) and float(izquierda.valor) == 1:
            return derecha
        if self.operador == '*' and isinstance(derecha, self.NodoNumero) and float(derecha.valor) == 0:
            return derecha
        if self.operador == '*' and isinstance(izquierda, self.NodoNumero) and float(izquierda.valor) == 0:
            return izquierda
        
        # Si no se puede optimizar mas, devolvemos la misma operacion
        return NodoOperacion(izquierda, self.operador, derecha)

In [425]:
class NodoRetorno(NodoAST):
    def __init__(self, expresion):
        self.expresion = expresion
    
    def traducir(self):
        return f"return {self.expresion.traducir()}"
    def to_dict(self):
        return {
            "tipo": "retorno",
            "expresion": self.expresion.to_dict()
        }

In [426]:
class NodoIdentificador(NodoAST):
    def __init__(self, nombre):
        self.nombre = nombre
        
    def to_dict(self):
        return {
            "tipo": "identificador",
            "nombre": self.nombre
        }
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        return self.nombre

In [427]:
class NodoNumero(NodoAST):
    def __init__(self, valor):
        self.valor = valor
        
    def to_dict(self):
        return {
            "tipo": "numero",
            "valor": self.valor
        }
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        return str(self.valor)

In [428]:
class NodoString(NodoAST):
    def __init__(self, valor):
        self.valor = valor
        
    def to_dict(self):
        return {
            "tipo": "string",
            "valor": self.valor
        }

In [429]:
class NodoIf(NodoAST):
    def __init__(self, condicion, cuerpo, cuerpo_else=None):
        self.condicion = condicion
        self.cuerpo = cuerpo
        self.cuerpo_else = cuerpo_else if cuerpo_else else []
        
    def to_dict(self):
        return {
            "tipo": "if",
            "condicion": self.condicion.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo],
            "else": [instr.to_dict() for instr in self.cuerpo_else]
        }
        
    def traducir(self):
        cuerpo_if = "\n    ".join(c.traducir() for c in self.cuerpo)
        if self.cuerpo_else:
            cuerpo_else = "\n    ".join(c.traducir() for c in self.cuerpo_else)
            return f"if {self.condicion.traducir()}:\n    {cuerpo_if}\nelse:\n    {cuerpo_else}"
        return f"if {self.condicion.traducir()}:\n    {cuerpo_if}"
        
    def optimizacion(self):
        # Optimizar la condición
        condicion_opt = self.condicion.optimizacion()
        
        # Optimizar los cuerpos
        cuerpo_opt = [inst.optimizacion() for inst in self.cuerpo if inst is not None]
        cuerpo_else_opt = [inst.optimizacion() for inst in self.cuerpo_else if inst is not None]
        
        # Si la condición es constante, podemos eliminar el if o el else
        if isinstance(condicion_opt, NodoNumero):
            if float(condicion_opt.valor) != 0:  # Condición siempre verdadera
                return cuerpo_opt
            else:  # Condición siempre falsa
                return cuerpo_else_opt
                
        return NodoIf(condicion_opt, cuerpo_opt, cuerpo_else_opt)

In [430]:
class NodoWhile(NodoAST):
    def __init__(self, condicion, cuerpo):
        self.condicion = condicion
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "while",
            "condicion": self.condicion.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [431]:
class NodoFor(NodoAST):
    def __init__(self, inicializacion, condicion, incremento, cuerpo):
        self.inicializacion = inicializacion
        self.condicion = condicion
        self.incremento = incremento
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "for",
            "inicializacion": self.inicializacion.to_dict(),
            "condicion": self.condicion.to_dict(),
            "incremento": self.incremento.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [432]:
class NodoIncremento(NodoAST):
    def __init__(self, nombre, operador):
        self.nombre = nombre
        self.operador = operador
        
    def to_dict(self):
        return {
            "tipo": "incremento",
            "nombre": self.nombre,
            "operador": self.operador
        }

In [433]:
class NodoPrint(NodoAST):
    def __init__(self, expresion):
        self.expresion = expresion
        
    def to_dict(self):
        return {
            "tipo": "print",
            "expresion": self.expresion.to_dict()
        }

In [434]:
class NodoElse(NodoAST):
    def __init__(self, cuerpo):
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "else",
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [435]:
class NodoLlamadaFuncion(NodoAST):
    def __init__(self, nombre, argumentos):
        self.nombre = nombre
        self.argumentos = argumentos
        
    def to_dict(self):
        return {
            "tipo": "llamada_funcion",
            "nombre": self.nombre,
            "argumentos": [arg.to_dict() for arg in self.argumentos]
        }

In [436]:
def ast_a_json(ast):
    return json.dumps(ast.to_dict(), indent=2)

In [437]:
# Función para imprimir el AST de forma legible
def imprimir_ast(nodo, nivel=0):
    prefijo = "  " * nivel
    
    if isinstance(nodo, NodoPrograma):
        print(f"{prefijo}Programa:")
        for funcion in nodo.funciones:
            imprimir_ast(funcion, nivel + 1)
        print(f"{prefijo}¿Tiene función main?: {'Sí' if nodo.tiene_main() else 'No'}")
        
    elif isinstance(nodo, NodoFuncion):
        print(f"{prefijo}Función: {nodo.nombre} (Tipo de retorno: {nodo.tipo})")
        print(f"{prefijo}Parámetros:")
        for param in nodo.parametros:
            imprimir_ast(param, nivel + 1)
        print(f"{prefijo}Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 1)
            
    elif isinstance(nodo, NodoParametro):
        print(f"{prefijo}Parámetro: {nodo.nombre} (Tipo: {nodo.tipo})")
        
    elif isinstance(nodo, NodoAsignacion):
        tipo_str = f"(Tipo: {nodo.tipo})" if nodo.tipo else ""
        print(f"{prefijo}Asignación: {nodo.nombre} {tipo_str}")
        print(f"{prefijo}  Valor:")
        imprimir_ast(nodo.expresion, nivel + 2)
        
    elif isinstance(nodo, NodoOperacion):
        print(f"{prefijo}Operación: {nodo.operador}")
        print(f"{prefijo}  Izquierda:")
        imprimir_ast(nodo.izquierda, nivel + 2)
        print(f"{prefijo}  Derecha:")
        imprimir_ast(nodo.derecha, nivel + 2)
        
    elif isinstance(nodo, NodoRetorno):
        print(f"{prefijo}Retorno:")
        imprimir_ast(nodo.expresion, nivel + 1)
        
    elif isinstance(nodo, NodoIdentificador):
        print(f"{prefijo}Identificador: {nodo.nombre}")
        
    elif isinstance(nodo, NodoNumero):
        print(f"{prefijo}Número: {nodo.valor}")
        
    elif isinstance(nodo, NodoString):
        print(f"{prefijo}String: {nodo.valor}")
        
    elif isinstance(nodo, NodoIf):
        print(f"{prefijo}If:")
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoWhile):
        print(f"{prefijo}While:")
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoFor):
        print(f"{prefijo}For:")
        print(f"{prefijo}  Inicialización:")
        imprimir_ast(nodo.inicializacion, nivel + 2)
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Incremento:")
        imprimir_ast(nodo.incremento, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoIncremento):
        print(f"{prefijo}Incremento: {nodo.nombre} {nodo.operador}")
        
    elif isinstance(nodo, NodoPrint):
        print(f"{prefijo}Print:")
        imprimir_ast(nodo.expresion, nivel + 1)
        
    elif isinstance(nodo, NodoElse):
        print(f"{prefijo}Else:")
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoLlamadaFuncion):
        print(f"{prefijo}Llamada a función: {nodo.nombre}")
        print(f"{prefijo}  Argumentos:")
        for arg in nodo.argumentos:
            imprimir_ast(arg, nivel + 2)
            
    else:
        print(f"{prefijo}Nodo desconocido: {type(nodo)}")

In [438]:
# Ejemplo de uso con múltiples funciones
texto_prueba = """
int suma(int a, int b) {
    return a + b;
}

int multiplica(int a, int b) {
    return a * b;
}

int main() {
    int resultado = 0;
    int x = 5;
    int y = 10;
    
    resultado = suma(x, y);
    print(resultado);
    
    resultado = multiplica(x, y);
    print(resultado);
    
    return 0;
}
"""

In [439]:
# Análisis léxico
tokens_ejercicio = identificar(texto_prueba)

In [440]:
try:
    # Análisis sintáctico y construcción del AST
    parser = Parser(tokens_ejercicio)
    ast_raiz = parser.parsear()
    
    print('Análisis sintáctico exitoso')
    
    # Convertir el AST a JSON y mostrarlo
    print("\n===== AST en formato JSON =====")
    json_ast = ast_a_json(ast_raiz)
    print(json_ast)
    
except SyntaxError as e:
    print(e)

Análisis sintáctico exitoso

===== AST en formato JSON =====
{
  "tipo": "programa",
  "funciones": [
    {
      "tipo": "funcion",
      "retorno": "int",
      "nombre": "suma",
      "parametros": [
        {
          "tipo": "parametro",
          "tipo_dato": "int",
          "nombre": "a"
        },
        {
          "tipo": "parametro",
          "tipo_dato": "int",
          "nombre": "b"
        }
      ],
      "cuerpo": [
        {
          "tipo": "retorno",
          "expresion": {
            "tipo": "operacion",
            "operador": "+",
            "izquierda": {
              "tipo": "identificador",
              "nombre": "a"
            },
            "derecha": {
              "tipo": "identificador",
              "nombre": "b"
            }
          }
        }
      ]
    },
    {
      "tipo": "funcion",
      "retorno": "int",
      "nombre": "multiplica",
      "parametros": [
        {
          "tipo": "parametro",
          "tipo_dato": "int",
   

In [441]:
codigo_fuente = '''
    int main(float a, int b){
        int c = a + b;
        return c;
    }'''

In [442]:
# Parse the codigo_fuente into tokens
tokens = identificar(codigo_fuente)

# Create a parser and generate the AST
parser_codigofuente = Parser(tokens)
arbol_ast = parser_codigofuente.parsear()

# Optimize the AST
arbol_ast_optimizado = arbol_ast.optimizacion()

# Translate the optimized AST
codigo_traducido = ""
for funcion in arbol_ast_optimizado.funciones:
    codigo_traducido += funcion.traducir() + "\n\n"

# Print the translated code
print("Código traducido y optimizado:")
print(codigo_traducido)

# Optionally, you can also print the original translated code for comparison
codigo_traducido_original = ""
for funcion in arbol_ast.funciones:
    codigo_traducido_original += funcion.traducir() + "\n\n"

print("\nCódigo traducido original (sin optimización):")
print(codigo_traducido_original)

Código traducido y optimizado:
def main(a.b):
    c = (a + b)
    return c



Código traducido original (sin optimización):
def main(a.b):
    c = (a + b)
    return c




In [443]:
class GeneradorEnsamblador:
    def __init__(self):
        self.codigo = []
        self.etiqueta_contador = 0
        self.tabla_simbolos = {}  # Ahora almacenará {'nombre': {'offset': int, 'tipo': str}}
        self.registros_enteros = ['eax', 'ebx', 'ecx', 'edx']
        self.registro_entero_actual = 0
        self.espacio_local = 0
        self.etiquetas_loop = []
        
    def nueva_etiqueta(self, prefijo="L"):
        self.etiqueta_contador += 1
        return f"{prefijo}_{self.etiqueta_contador}"
        
    def obtener_tipo(self, valor):
        """Determina si un valor es entero o flotante"""
        if isinstance(valor, str):
            if '.' in valor:
                return 'float'
            return 'int'
        return 'int' if isinstance(valor, int) else 'float'
    
    def generar(self, nodo):
        if isinstance(nodo, NodoPrograma):
            self.codigo.append("section .text")
            self.codigo.append("global _start")
            self.codigo.append("_start:")
            self.codigo.append("  call main")
            self.codigo.append("  mov ebx, eax")  # Usar resultado como código de salida
            self.codigo.append("  mov eax, 1")    # sys_exit
            self.codigo.append("  int 0x80")
            self.codigo.append("")
            
            for funcion in nodo.funciones:
                self.generar(funcion)
                
        elif isinstance(nodo, NodoFuncion):
            # Reiniciar estado para cada función
            self.tabla_simbolos = {}
            self.espacio_local = 0
            
            # Prologo de la funcion
            self.codigo.append(f"{nodo.nombre}:")
            self.codigo.append("  push ebp")
            self.codigo.append("  mov ebp, esp")
            
            # Calcular espacio necesario para variables locales
            for instruccion in nodo.cuerpo:
                if isinstance(instruccion, NodoAsignacion) and instruccion.tipo:
                    tipo = instruccion.tipo.lower()  # 'int' o 'float'
                    self.tabla_simbolos[instruccion.nombre] = {
                        'offset': self.espacio_local + 4,
                        'tipo': tipo
                    }
                    self.espacio_local += 4
                    
            if self.espacio_local > 0:
                self.codigo.append(f"  sub esp, {self.espacio_local}")
                
            # Generar parámetros (se pasan en la pila)
            for i, param in enumerate(nodo.parametros):
                self.tabla_simbolos[param.nombre] = {
                    'offset': 8 + i * 4,  # ebp+8 es el primer parámetro
                    'tipo': param.tipo.lower()
                }
                
            # Generar cuerpo de la función
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            # Si no hay return, agregar uno por defecto
            if not any(isinstance(i, NodoRetorno) for i in nodo.cuerpo):
                if nodo.tipo != "void":
                    self.codigo.append("  mov eax, 0")  # Valor de retorno por defecto
                self.codigo.append("  mov esp, ebp")
                self.codigo.append("  pop ebp")
                self.codigo.append("  ret")
                
        elif isinstance(nodo, NodoIf):
            # Generar la condición
            self.generar(nodo.condicion)
            
            # Crear etiquetas
            etiqueta_else = self.nueva_etiqueta("else")
            etiqueta_fin = self.nueva_etiqueta("endif")
            
            # Saltar si la condición es falsa (0)
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_else}")
            
            # Código del if (cuerpo)
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
            
            # Saltar al final para evitar el else
            self.codigo.append(f"  jmp {etiqueta_fin}")
            
            # Código del else (si existe)
            self.codigo.append(f"{etiqueta_else}:")
            for instruccion in nodo.cuerpo_else:
                self.generar(instruccion)
            
            # Etiqueta de fin
            self.codigo.append(f"{etiqueta_fin}:")
            
        elif isinstance(nodo, NodoWhile):
            etiqueta_inicio = self.nueva_etiqueta("while_start")
            etiqueta_fin = self.nueva_etiqueta("while_end")
            
            # Guardar etiquetas para break/continue
            self.etiquetas_loop.append((etiqueta_fin, etiqueta_inicio))
            
            self.codigo.append(f"{etiqueta_inicio}:")
            
            # Generar condición
            self.generar(nodo.condicion)
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_fin}")
            
            # Cuerpo del while
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            self.codigo.append(f"  jmp {etiqueta_inicio}")
            self.codigo.append(f"{etiqueta_fin}:")
            
            # Sacar etiquetas del stack
            self.etiquetas_loop.pop()
            
        elif isinstance(nodo, NodoFor):
            etiqueta_inicio = self.nueva_etiqueta("for_start")
            etiqueta_fin = self.nueva_etiqueta("for_end")
            etiqueta_incremento = self.nueva_etiqueta("for_inc")
            
            # Inicialización
            self.generar(nodo.inicializacion)
            
            # Guardar etiquetas para break/continue
            self.etiquetas_loop.append((etiqueta_fin, etiqueta_incremento))
            
            self.codigo.append(f"{etiqueta_inicio}:")
            
            # Condición
            self.generar(nodo.condicion)
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_fin}")
            
            # Cuerpo
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            # Incremento
            self.codigo.append(f"{etiqueta_incremento}:")
            self.generar(nodo.incremento)
            
            self.codigo.append(f"  jmp {etiqueta_inicio}")
            self.codigo.append(f"{etiqueta_fin}:")
            
            # Sacar etiquetas del stack
            self.etiquetas_loop.pop()
            
        elif isinstance(nodo, NodoAsignacion):
            # Generar la expresión
            self.generar(nodo.expresion)
            
            # Determinar el tipo de la variable
            tipo_var = None
            if nodo.tipo:  # Declaración con tipo explícito
                tipo_var = nodo.tipo.lower()
            elif nodo.nombre in self.tabla_simbolos:  # Variable existente
                tipo_var = self.tabla_simbolos[nodo.nombre]['tipo']
            else:  # Inferir tipo de la expresión
                if isinstance(nodo.expresion, NodoNumero):
                    tipo_var = self.obtener_tipo(nodo.expresion.valor)
                elif isinstance(nodo.expresion, NodoIdentificador):
                    if nodo.expresion.nombre in self.tabla_simbolos:
                        tipo_var = self.tabla_simbolos[nodo.expresion.nombre]['tipo']
                else:
                    tipo_var = 'int'  # Por defecto asumimos entero
            
            # Guardar el resultado
            if nodo.nombre in self.tabla_simbolos:
                offset = self.tabla_simbolos[nodo.nombre]['offset']
                if tipo_var == 'float':
                    self.codigo.append("  fstp dword [ebp - {offset}]")  # Guardar float
                else:
                    self.codigo.append(f"  mov [ebp - {offset}], eax")  # Guardar int
            else:
                # Es una variable local nueva
                self.espacio_local += 4
                offset = self.espacio_local
                self.tabla_simbolos[nodo.nombre] = {
                    'offset': offset,
                    'tipo': tipo_var if tipo_var else 'int'  # Por defecto int
                }
                if tipo_var == 'float':
                    self.codigo.append(f"  fstp dword [ebp - {offset}]")  # Guardar float
                else:
                    self.codigo.append(f"  mov [ebp - {offset}], eax")  # Guardar int
                
        elif isinstance(nodo, NodoOperacion):
            # Determinar tipos de los operandos
            tipo_izq = self.determinar_tipo(nodo.izquierda)
            tipo_der = self.determinar_tipo(nodo.derecha)
            
            # Manejar operaciones entre diferentes tipos (conversión implícita)
            if tipo_izq != tipo_der:
                # Convertir el entero a flotante si es necesario
                if tipo_izq == 'int' and tipo_der == 'float':
                    self.generar_conversion(nodo.izquierda, 'int', 'float')
                    tipo_izq = 'float'
                elif tipo_izq == 'float' and tipo_der == 'int':
                    self.generar_conversion(nodo.derecha, 'int', 'float')
                    tipo_der = 'float'
            
            # Generar código según el tipo
            if tipo_izq == 'float':
                self.generar_operacion_float(nodo)
            else:
                self.generar_operacion_int(nodo)
                
        elif isinstance(nodo, NodoRetorno):
            if nodo.expresion:
                self.generar(nodo.expresion)
            else:
                self.codigo.append("  mov eax, 0")  # Valor de retorno por defecto
                
            self.codigo.append("  mov esp, ebp")
            self.codigo.append("  pop ebp")
            self.codigo.append("  ret")
            
        elif isinstance(nodo, NodoNumero):
            tipo = self.obtener_tipo(nodo.valor)
            if tipo == 'float':
                # Almacenar el float en memoria y cargarlo en FPU
                etiqueta = self.nueva_etiqueta("float_const")
                self.codigo.append(f"  mov eax, {etiqueta}")
                self.codigo.append(f"  fld dword [eax]")
                
                # Agregar a la sección de datos
                if not hasattr(self, 'data_section'):
                    self.data_section = []
                self.data_section.append(f"{etiqueta} dd {nodo.valor}")
            else:
                self.codigo.append(f"  mov eax, {nodo.valor}")
            
        elif isinstance(nodo, NodoIdentificador):
            var_info = self.tabla_simbolos[nodo.nombre]
            offset = var_info['offset']
            
            if var_info['tipo'] == 'float':
                if offset > 0:  # Variable local
                    self.codigo.append(f"  fld dword [ebp - {offset}]")
                else:  # Parámetro
                    self.codigo.append(f"  fld dword [ebp + {8 + offset}]")
            else:
                if offset > 0:  # Variable local
                    self.codigo.append(f"  mov eax, [ebp - {offset}]")
                else:  # Parámetro
                    self.codigo.append(f"  mov eax, [ebp + {8 + offset}]")
                    
        elif isinstance(nodo, NodoLlamadaFuncion):
            # Empujar argumentos en la pila (en orden inverso)
            for arg in reversed(nodo.argumentos):
                self.generar(arg)
                
                # Verificar si es float para manejarlo adecuadamente
                tipo_arg = self.determinar_tipo(arg)
                if tipo_arg == 'float':
                    self.codigo.append("  sub esp, 4")
                    self.codigo.append("  fstp dword [esp]")
                else:
                    self.codigo.append("  push eax")
                
            self.codigo.append(f"  call {nodo.nombre}")
            
            # Limpiar la pila de argumentos
            if nodo.argumentos:
                self.codigo.append(f"  add esp, {len(nodo.argumentos) * 4}")
                
        elif isinstance(nodo, NodoPrint):
            if isinstance(nodo.expresion, NodoString):
                # Manejo de strings como antes
                if not hasattr(self, 'data_section'):
                    self.data_section = []
                    self.string_labels = {}
                
                if nodo.expresion.valor not in self.string_labels:
                    label = self.nueva_etiqueta("str")
                    self.string_labels[nodo.expresion.valor] = label
                    self.data_section.append(f'{label} db {nodo.expresion.valor}, 10')
                    self.data_section.append(f'{label}_len equ $ - {label}')
                
                self.codigo.append(f"  mov eax, 4")          # sys_write
                self.codigo.append(f"  mov ebx, 1")          # stdout
                self.codigo.append(f"  mov ecx, {self.string_labels[nodo.expresion.valor]}")
                self.codigo.append(f"  mov edx, {self.string_labels[nodo.expresion.valor]}_len")
                self.codigo.append(f"  int 0x80")
            else:
                # Para números (enteros o flotantes)
                tipo = self.determinar_tipo(nodo.expresion)
                self.generar(nodo.expresion)
                
                if tipo == 'float':
                    # Convertir float a string e imprimir (implementación simplificada)
                    self.codigo.append("  ; Conversión de float a string (simplificada)")
                    self.codigo.append("  sub esp, 8")
                    self.codigo.append("  fstp qword [esp]")
                    self.codigo.append("  push format_float")
                    self.codigo.append("  call printf")
                    self.codigo.append("  add esp, 12")
                    
                    # Agregar format_float a la sección de datos
                    if not hasattr(self, 'data_section'):
                        self.data_section = []
                    if "format_float db '%.2f', 10, 0" not in self.data_section:
                        self.data_section.append("format_float db '%.2f', 10, 0")
                else:
                    # Convertir entero a string e imprimir
                    self.codigo.append("  ; Conversión de entero a string e impresión")
                    self.codigo.append("  push eax")
                    self.codigo.append("  push format_int")
                    self.codigo.append("  call printf")
                    self.codigo.append("  add esp, 8")
                    
                    # Agregar format_int a la sección de datos
                    if not hasattr(self, 'data_section'):
                        self.data_section = []
                    if "format_int db '%d', 10, 0" not in self.data_section:
                        self.data_section.append("format_int db '%d', 10, 0")
            
        elif isinstance(nodo, NodoIncremento):
            var_info = self.tabla_simbolos[nodo.nombre]
            offset = var_info['offset']
            
            if var_info['tipo'] == 'float':
                if offset > 0:
                    self.codigo.append(f"  fld dword [ebp - {offset}]")
                else:
                    self.codigo.append(f"  fld dword [ebp + {8 + offset}]")
                
                if nodo.operador == '++':
                    self.codigo.append("  fld1")
                    self.codigo.append("  faddp")
                else:
                    self.codigo.append("  fld1")
                    self.codigo.append("  fsubp")
                
                if offset > 0:
                    self.codigo.append(f"  fstp dword [ebp - {offset}]")
                else:
                    self.codigo.append(f"  fstp dword [ebp + {8 + offset}]")
            else:
                if offset > 0:
                    self.codigo.append(f"  mov eax, [ebp - {offset}]")
                else:
                    self.codigo.append(f"  mov eax, [ebp + {8 + offset}]")
                
                if nodo.operador == '++':
                    self.codigo.append("  inc eax")
                else:
                    self.codigo.append("  dec eax")
                
                if offset > 0:
                    self.codigo.append(f"  mov [ebp - {offset}], eax")
                else:
                    self.codigo.append(f"  mov [ebp + {8 + offset}], eax")
            
        else:
            raise ValueError(f"Nodo no soportado: {type(nodo)}")
    
    def determinar_tipo(self, nodo):
        """Determina el tipo de un nodo de expresión"""
        if isinstance(nodo, NodoNumero):
            return self.obtener_tipo(nodo.valor)
        elif isinstance(nodo, NodoIdentificador):
            if nodo.nombre in self.tabla_simbolos:
                return self.tabla_simbolos[nodo.nombre]['tipo']
            return 'int'  # Por defecto
        elif isinstance(nodo, NodoOperacion):
            # Para operaciones, asumimos que ya están balanceadas por generar_operacion
            tipo_izq = self.determinar_tipo(nodo.izquierda)
            tipo_der = self.determinar_tipo(nodo.derecha)
            return 'float' if 'float' in (tipo_izq, tipo_der) else 'int'
        elif isinstance(nodo, NodoLlamadaFuncion):
            # Asumimos que las funciones devuelven int por defecto
            # En una implementación real, necesitaríamos información de tipos de funciones
            return 'int'
        return 'int'  # Por defecto
    
    def generar_conversion(self, nodo, tipo_origen, tipo_destino):
        """Genera código para conversión de tipos"""
        if tipo_origen == 'int' and tipo_destino == 'float':
            # Convertir entero a flotante
            self.generar(nodo)
            self.codigo.append("  push eax")
            self.codigo.append("  fild dword [esp]")
            self.codigo.append("  add esp, 4")
        elif tipo_origen == 'float' and tipo_destino == 'int':
            # Convertir flotante a entero (truncando)
            self.generar(nodo)
            self.codigo.append("  sub esp, 4")
            self.codigo.append("  fistp dword [esp]")
            self.codigo.append("  pop eax")
        else:
            raise ValueError(f"Conversión no soportada: {tipo_origen} a {tipo_destino}")
    
    def generar_operacion_int(self, nodo):
        """Genera código para operaciones con enteros"""
        self.generar(nodo.izquierda)
        self.codigo.append("  push eax")
        self.generar(nodo.derecha)
        self.codigo.append("  pop ebx")
        
        if nodo.operador == '+':
            self.codigo.append("  add eax, ebx")
        elif nodo.operador == '-':
            self.codigo.append("  sub eax, ebx")
        elif nodo.operador == '*':
            self.codigo.append("  imul eax, ebx")
        elif nodo.operador == '/':
            self.codigo.append("  cdq")
            self.codigo.append("  idiv ebx")
        elif nodo.operador == '==':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  sete al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '!=':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  setne al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '<':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  setl al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '>':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  setg al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '<=':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  setle al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '>=':
            self.codigo.append("  cmp ebx, eax")
            self.codigo.append("  setge al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '&&':
            self.codigo.append("  and eax, ebx")
            self.codigo.append("  cmp eax, 0")
            self.codigo.append("  setne al")
            self.codigo.append("  movzx eax, al")
        elif nodo.operador == '||':
            self.codigo.append("  or eax, ebx")
            self.codigo.append("  cmp eax, 0")
            self.codigo.append("  setne al")
            self.codigo.append("  movzx eax, al")
    
    def generar_operacion_float(self, nodo):
        """Genera código para operaciones con flotantes"""
        self.generar(nodo.izquierda)
        self.generar(nodo.derecha)
        
        if nodo.operador == '+':
            self.codigo.append("  faddp")
        elif nodo.operador == '-':
            self.codigo.append("  fsubp")
        elif nodo.operador == '*':
            self.codigo.append("  fmulp")
        elif nodo.operador == '/':
            self.codigo.append("  fdivp")
        elif nodo.operador in ('==', '!=', '<', '>', '<=', '>='):
            # Comparación de flotantes
            self.codigo.append("  fcomip st0, st1")
            self.codigo.append("  fstp st0")  # Limpiar pila FPU
            
            if nodo.operador == '==':
                self.codigo.append("  sete al")
            elif nodo.operador == '!=':
                self.codigo.append("  setne al")
            elif nodo.operador == '<':
                self.codigo.append("  setb al")
            elif nodo.operador == '>':
                self.codigo.append("  seta al")
            elif nodo.operador == '<=':
                self.codigo.append("  setbe al")
            elif nodo.operador == '>=':
                self.codigo.append("  setae al")
                
            self.codigo.append("  movzx eax, al")
        else:
            raise ValueError(f"Operador no soportado para flotantes: {nodo.operador}")
    
    def obtener_codigo(self):
        # Construir el código completo con todas las secciones necesarias
        sections = []
        
        # Sección .data si hay strings o constantes
        if hasattr(self, 'data_section'):
            sections.append("section .data")
            sections.extend(self.data_section)
        
        # Sección .bss si hay buffers
        if hasattr(self, 'bss_section'):
            sections.append("section .bss")
            sections.extend(self.bss_section)
        
        # Sección .text (siempre presente)
        sections.append("section .text")
        sections.append("global _start")
        sections.append("extern printf")  # Necesario para las funciones de impresión
        sections.extend(self.codigo)
        
        # Filtrar líneas vacías innecesarias
        filtered = []
        for line in sections:
            if line.strip() or (not filtered or filtered[-1].strip()):
                filtered.append(line)
        
        return "\n".join(filtered)

In [444]:
class Optimizador:
    def optimizar(self, nodo):
        if isinstance(nodo, NodoPrograma):
            nodo.funciones = [self.optimizar(funcion) for funcion in nodo.funciones]
            return nodo
            
        elif isinstance(nodo, NodoFuncion):
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                instr_opt = self.optimizar(instr)
                if instr_opt is not None:  # Puede ser None si se eliminó
                    if isinstance(instr_opt, list):  # Algunas optimizaciones devuelven listas
                        nuevo_cuerpo.extend(instr_opt)
                    else:
                        nuevo_cuerpo.append(instr_opt)
            nodo.cuerpo = nuevo_cuerpo
            return nodo
            
        elif isinstance(nodo, NodoOperacion):
            # Optimizar subexpresiones
            nodo.izquierda = self.optimizar(nodo.izquierda)
            nodo.derecha = self.optimizar(nodo.derecha)
            
            # Plegado de constantes
            if isinstance(nodo.izquierda, NodoNumero) and isinstance(nodo.derecha, NodoNumero):
                try:
                    if nodo.operador == '+':
                        return NodoNumero(float(nodo.izquierda.valor) + float(nodo.derecha.valor))
                    elif nodo.operador == '-':
                        return NodoNumero(float(nodo.izquierda.valor) - float(nodo.derecha.valor))
                    elif nodo.operador == '*':
                        return NodoNumero(float(nodo.izquierda.valor) * float(nodo.derecha.valor))
                    elif nodo.operador == '/' and float(nodo.derecha.valor) != 0:
                        return NodoNumero(float(nodo.izquierda.valor) / float(nodo.derecha.valor))
                    elif nodo.operador == '==':
                        return NodoNumero(1 if float(nodo.izquierda.valor) == float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '!=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) != float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '<':
                        return NodoNumero(1 if float(nodo.izquierda.valor) < float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '>':
                        return NodoNumero(1 if float(nodo.izquierda.valor) > float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '<=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) <= float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '>=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) >= float(nodo.derecha.valor) else 0)
                except:
                    pass  # Si hay error en la conversión, mantener la operación
                    
            # Simplificaciones algebraicas
            if nodo.operador == '+' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 0:
                return nodo.izquierda
            if nodo.operador == '*' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 1:
                return nodo.izquierda
            if nodo.operador == '*' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 0:
                return NodoNumero(0)
            if nodo.operador == '&&' and isinstance(nodo.izquierda, NodoNumero) and float(nodo.izquierda.valor) == 0:
                return NodoNumero(0)
            if nodo.operador == '||' and isinstance(nodo.izquierda, NodoNumero) and float(nodo.izquierda.valor) != 0:
                return NodoNumero(1)
                
            return nodo
            
        elif isinstance(nodo, NodoIf):
            nodo.condicion = self.optimizar(nodo.condicion)
            
            # Optimizar cuerpos
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            nuevo_cuerpo_else = []
            for instr in nodo.cuerpo_else:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo_else.append(opt_instr)
            nodo.cuerpo_else = nuevo_cuerpo_else
            
            # Si la condición es constante
            if isinstance(nodo.condicion, NodoNumero):
                if float(nodo.condicion.valor) != 0:
                    return nodo.cuerpo  # Devolver solo el cuerpo del if
                else:
                    return nodo.cuerpo_else  # Devolver solo el cuerpo del else
                    
            # Si el cuerpo del if está vacío y no hay else
            if not nodo.cuerpo and not nodo.cuerpo_else:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoWhile):
            nodo.condicion = self.optimizar(nodo.condicion)
            
            # Optimizar cuerpo
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            # Si la condición es falsa constante, eliminar el while
            if isinstance(nodo.condicion, NodoNumero) and float(nodo.condicion.valor) == 0:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoFor):
            nodo.inicializacion = self.optimizar(nodo.inicializacion)
            nodo.condicion = self.optimizar(nodo.condicion)
            nodo.incremento = self.optimizar(nodo.incremento)
            
            # Optimizar cuerpo
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            
        elif isinstance(nodo, NodoAsignacion):
            nodo.expresion = self.optimizar(nodo.expresion)
            
            # Eliminar asignaciones redundantes (x = x)
            if isinstance(nodo.expresion, NodoIdentificador) and nodo.nombre == nodo.expresion.nombre:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoPrint):
            # Asegurarnos de que existe la sección bss
            if not hasattr(self, 'bss_section'):
                self.bss_section = []
            
            # Agregar buffer si no existe
            if "buffer resb 12" not in self.bss_section:
                self.bss_section.append("buffer resb 12")
                    
            nodo.expresion = self.optimizar(nodo.expresion)
            return nodo
            
        elif isinstance(nodo, (NodoRetorno, NodoIdentificador, NodoNumero, NodoString, 
                                NodoLlamadaFuncion, NodoIncremento, NodoElse)):
            # Nodos que no necesitan optimización especial
            return nodo
            
        else:
            raise ValueError(f"Tipo de nodo no reconocido: {type(nodo)}")

In [445]:
class GeneradorBinario:
    def __init__(self):
        self.ensamblador = GeneradorEnsamblador()
        
    def compilar(self, ast, archivo_salida="salida"):
        # 1. Optimizar el AST
        optimizador = Optimizador()
        ast_optimizado = optimizador.optimizar(ast)
        
        # 2. Generar código ensamblador
        self.ensamblador.generar(ast_optimizado)
        codigo_asm = self.ensamblador.obtener_codigo()
        
        # 3. Escribir archivo .asm
        with open(f"{archivo_salida}.asm", "w") as f:
            f.write(codigo_asm)
            print(f"Archivo {archivo_salida}.asm generado correctamente")
            
        try:
            print("Ensamblando con NASM...")
            
            try:
                subprocess.run(["nasm", "-f", "win32", f"{archivo_salida}.asm", "-o", f"{archivo_salida}.obj"], check=True, capture_output=True, text=True)
                print("Ensamblado completado")
            except subprocess.CalledProcessError as e:
                print("Error al ejecutar NASM:")
                print("STDOUT:", e.stdout)
                print("STDERR:", e.stderr)
                return False
            
            print("Enlazando con gcc...")
            
            subprocess.run(["gcc", f"{archivo_salida}.obj", "-o", f"{archivo_salida}.exe", "-nostartfiles", "-Wl,-e,_start"], check=True)

            print("Enlazado completado")
            
            print(f"Compilación exitosa. Ejecutable generado: {archivo_salida}.exe")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error durante la compilación (código {e.returncode}): {e}")
            if e.stdout:
                print("Salida estándar:", e.stdout.decode())
            if e.stderr:
                print("Error estándar:", e.stderr.decode())
            return False
        except FileNotFoundError:
            print("Error: NASM o link.exe no están instalados o no están en el PATH")
            print("Asegúrate de que:")
            print("1. NASM esté instalado y en el PATH")
            print("2. Visual Studio o las herramientas de compilación de C++ estén instaladas")
            return False
            
    @staticmethod
    def ejecutar_en_nueva_ventana(archivo_exe):
        try:
            subprocess.run([archivo_exe], shell=True, check=True)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error al ejecutar: {e}")
            return False

In [446]:
# Código fuente de ejemplo
codigo_fuente = """
int suma(int a, int b) {
    int result = a + b;
    return result;
}

int main() {
    int num1 = 2;
    int num2 = 2;
    int resultado = suma(num1, num2);
    print(resultado);
    return 0;
}
"""

# 1. Análisis léxico
tokens = identificar(codigo_fuente)

# 2. Análisis sintáctico
parser = Parser(tokens)
ast = parser.parsear()

# 3. Optimización
optimizador = Optimizador()
ast_optimizado = optimizador.optimizar(ast)

# 4. Generación de código
generador_bin = GeneradorBinario()
exito = generador_bin.compilar(ast_optimizado, "factorial")

# 5. Mostrar código ensamblador generado
if exito:
    print("\nCódigo ensamblador generado:")
    print(generador_bin.ensamblador.obtener_codigo())
    
# Uso en cmd
GeneradorBinario.ejecutar_en_nueva_ventana("factorial.exe")

Archivo factorial.asm generado correctamente
Ensamblando con NASM...
Ensamblado completado
Enlazando con gcc...
Error durante la compilación (código 1): Command '['gcc', 'factorial.obj', '-o', 'factorial.exe', '-nostartfiles', '-Wl,-e,_start']' returned non-zero exit status 1.
Error al ejecutar: Command '['factorial.exe']' returned non-zero exit status 1.


False

# Analizador Semántico

In [447]:
class AnalizadorSemantico:
    def __init__(self):
        self.tabla_simbolos = {}
        self.funcion_actual = None
        
    def analizar(self, nodo):
        metodo = f'visitar_{type(nodo).__name__}'
        if hasattr(self, metodo):
            return getattr(self, metodo)(nodo)
        else:
            # Para nodos que no necesitan análisis semántico especial
            return None
        
    def visitar_NodoFuncion(self, nodo):
        if nodo.nombre in self.tabla_simbolos:
            raise Exception(f'Error semántico: la función {nodo.nombre} ya está definida')
            
        # Registrar la función en la tabla de símbolos
        self.tabla_simbolos[nodo.nombre] = {
            'tipo': nodo.tipo,
            'parametros': [(p.tipo, p.nombre) for p in nodo.parametros],
            'es_funcion': True
        }
        
        self.funcion_actual = nodo.nombre
        
        # Registrar parámetros
        for param in nodo.parametros:
            self.tabla_simbolos[param.nombre] = {
                'tipo': param.tipo,
                'es_parametro': True
            }
            
        # Analizar cuerpo de la función
        for instruccion in nodo.cuerpo:
            self.analizar(instruccion)
            
        self.funcion_actual = None
        
    def visitar_NodoAsignacion(self, nodo):
        tipo_expresion = self.analizar(nodo.expresion)
        
        # Si es una declaración (tiene tipo)
        if nodo.tipo:
            if nodo.tipo != tipo_expresion:
                raise Exception(f'Error semántico: Tipo de expresión ({tipo_expresion}) no coincide con declaración ({nodo.tipo})')
        
        # Registrar o verificar variable
        if nodo.nombre in self.tabla_simbolos:
            if self.tabla_simbolos[nodo.nombre].get('es_parametro', False):
                raise Exception(f'Error semántico: No se puede redefinir el parámetro {nodo.nombre}')
            if tipo_expresion != self.tabla_simbolos[nodo.nombre]['tipo']:
                raise Exception(f'Error semántico: Tipo de asignación no coincide con declaración previa de {nodo.nombre}')
        else:
            # Si no está declarada, la registramos con el tipo de la expresión
            self.tabla_simbolos[nodo.nombre] = {
                'tipo': tipo_expresion,
                'es_variable': True
            }
            
    def visitar_NodoOperacion(self, nodo):
        tipo_izquierda = self.analizar(nodo.izquierda)
        tipo_derecha = self.analizar(nodo.derecha)
        
        # Verificar compatibilidad de tipos
        if tipo_izquierda != tipo_derecha:
            raise Exception(f'Error semántico: Operación {nodo.operador} entre tipos incompatibles: {tipo_izquierda} y {tipo_derecha}')
        
        # El tipo resultante de la operación es el tipo de los operandos
        return tipo_izquierda
    
    def visitar_NodoNumero(self, nodo):
        return 'int' if '.' not in nodo.valor else 'float'
    
    def visitar_NodoIdentificador(self, nodo):
        if nodo.nombre not in self.tabla_simbolos:
            raise Exception(f'Error semántico: La variable {nodo.nombre} no está declarada')
        return self.tabla_simbolos[nodo.nombre]['tipo']
    
    def visitar_NodoPrograma(self, nodo):
        for funcion in nodo.funciones:
            self.analizar(funcion)
            
        # Verificar que exista la función main
        if 'main' not in self.tabla_simbolos:
            raise Exception('Error semántico: No se encontró la función main()')
            
    def visitar_NodoRetorno(self, nodo):
        tipo_expresion = self.analizar(nodo.expresion)
        
        # Verificar que el tipo de retorno coincida con el de la función
        if self.funcion_actual:
            tipo_funcion = self.tabla_simbolos[self.funcion_actual]['tipo']
            if tipo_expresion != tipo_funcion:
                raise Exception(f'Error semántico: Tipo de retorno ({tipo_expresion}) no coincide con el tipo de la función ({tipo_funcion})')
        
        return tipo_expresion
    
    def visitar_NodoLlamadaFuncion(self, nodo):
        if nodo.nombre not in self.tabla_simbolos:
            raise Exception(f'Error semántico: Función {nodo.nombre} no está declarada')
            
        info_funcion = self.tabla_simbolos[nodo.nombre]
        
        # Verificar número de parámetros
        if len(nodo.argumentos) != len(info_funcion['parametros']):
            raise Exception(f'Error semántico: Número incorrecto de argumentos para {nodo.nombre}')
            
        # Verificar tipos de parámetros
        for arg, (tipo_param, _) in zip(nodo.argumentos, info_funcion['parametros']):
            tipo_arg = self.analizar(arg)
            if tipo_arg != tipo_param:
                raise Exception(f'Error semántico: Tipo de argumento {tipo_arg} no coincide con parámetro {tipo_param}')
                
        return info_funcion['tipo']

# Prueba

In [448]:

##codigo_fuente =
##   int suma(int a, int b) {
##        int c = a + b;
##        return c;
##}

analizador_semantico = AnalizadorSemantico()
analisis = analizador_semantico.analizar(arbol_ast)

Exception: Error semántico: Operación + entre tipos incompatibles: float y int

In [ ]:
analizador_semantico.tabla_simbolos

{'main': {'tipo': 'int',
  'parametros': [('float', 'a'), ('int', 'b')],
  'es_funcion': True},
 'a': {'tipo': 'float', 'es_parametro': True},
 'b': {'tipo': 'int', 'es_parametro': True}}

In [ ]:
for llave in (analizador_semantico.tabla_simbolos.keys()):
    valor = analizador_semantico.tabla_simbolos.get(llave)
    print(f'{llave} {valor}')

main {'tipo': 'int', 'parametros': [('float', 'a'), ('int', 'b')], 'es_funcion': True}
a {'tipo': 'float', 'es_parametro': True}
b {'tipo': 'int', 'es_parametro': True}


In [ ]:
class TablaSimbolos:
    def __init__(self, padre=None):
        self.padre = padre  # anidados
        self.variables = {}
        self.funciones = {}
        
    def declarar_variable(self, nombre, tipo):
        if nombre in self.variables:
            raise Exception(f'Error semántico: Variable "{nombre}" ya declarada')
        self.variables[nombre] = {'tipo': tipo, 'es_variable': True}
        
    def obtener_tipo_variable(self, nombre):
        if nombre in self.variables:
            return self.variables[nombre]['tipo']
        elif self.padre:
            return self.padre.obtener_tipo_variable(nombre)
        raise Exception(f'Error semántico: Variable "{nombre}" no declarada')
    
    def declarar_funcion(self, nombre, tipo_retorno, parametros):
        if nombre in self.funciones:
            raise Exception(f'Error semántico: Función "{nombre}" ya declarada')
        self.funciones[nombre] = {
            'tipo_retorno': tipo_retorno,
            'parametros': parametros,
            'es_funcion': True
        }
    
    def obtener_funcion(self, nombre):
        if nombre in self.funciones:
            return self.funciones[nombre]
        elif self.padre:
            return self.padre.obtener_funcion(nombre)
        raise Exception(f'Error semántico: Función "{nombre}" no declarada')

In [ ]:
class AnalizadorSemantico:
    def __init__(self):
        self.tabla_global = TablaSimbolos()
        self.tabla_actual = self.tabla_global
        self.funcion_actual = None
        
    def analizar(self, nodo):
        metodo = f'analizar_{type(nodo).__name__}'
        if hasattr(self, metodo):
            return getattr(self, metodo)(nodo)
        else:
            return None
    
    def analizar_NodoPrograma(self, nodo):
        for funcion in nodo.funciones:
            self.analizar(funcion)
        
        if 'main' not in self.tabla_global.funciones:
            raise Exception('Error semántico: No se encontró la función main()')
    
    def analizar_NodoFuncion(self, nodo):
        self.tabla_global.declarar_funcion(
            nodo.nombre,
            nodo.tipo,
            [(p.tipo, p.nombre) for p in nodo.parametros]
        )
        
        ambito_funcion = TablaSimbolos(padre=self.tabla_global)
        self.tabla_actual = ambito_funcion
        self.funcion_actual = nodo.nombre
        
        for param in nodo.parametros:
            self.tabla_actual.declarar_variable(param.nombre, param.tipo)
        
        for instruccion in nodo.cuerpo:
            self.analizar(instruccion)
    
        self.tabla_actual = self.tabla_global
        self.funcion_actual = None
    
    def analizar_NodoAsignacion(self, nodo):
        tipo_expresion = self.analizar(nodo.expresion)
    
        if nodo.tipo:
            if nodo.tipo != tipo_expresion:
                raise Exception(
                    f'Error semántico: Tipo de expresión ({tipo_expresion}) '
                    f'no coincide con la declaración ({nodo.tipo}) para "{nodo.nombre}"'
                )
            self.tabla_actual.declarar_variable(nodo.nombre, nodo.tipo)
        else:
            tipo_variable = self.tabla_actual.obtener_tipo_variable(nodo.nombre)
            if tipo_variable != tipo_expresion:
                raise Exception(
                    f'Error semántico: Tipo de asignación ({tipo_expresion}) '
                    f'no coincide con el tipo de la variable ({tipo_variable})'
                )
    
    def analizar_NodoIdentificador(self, nodo):
        return self.tabla_actual.obtener_tipo_variable(nodo.nombre)
    
    def analizar_NodoNumero(self, nodo):
        return 'int' if '.' not in nodo.valor else 'float'
    
    def analizar_NodoOperacion(self, nodo):
        tipo_izquierda = self.analizar(nodo.izquierda)
        tipo_derecha = self.analizar(nodo.derecha)
        
        if tipo_izquierda != tipo_derecha:
            raise Exception(
                f'Error semántico: Operación {nodo.operador} entre tipos '
                f'incompatibles: {tipo_izquierda} y {tipo_derecha}'
            )
        return tipo_izquierda
    
    def analizar_NodoRetorno(self, nodo):
        if not self.funcion_actual:
            raise Exception('Error semántico: return fuera de función')
        
        tipo_expresion = self.analizar(nodo.expresion)
        tipo_funcion = self.tabla_global.obtener_funcion(self.funcion_actual)['tipo_retorno']
        
        if tipo_expresion != tipo_funcion:
            raise Exception(
                f'Error semántico: Tipo de retorno ({tipo_expresion}) '
                f'no coincide con el tipo de la función ({tipo_funcion})'
            )
    
    def analizar_NodoLlamadaFuncion(self, nodo):
        try:
            info_funcion = self.tabla_global.obtener_funcion(nodo.nombre)
        except Exception as e:
            raise Exception(f'Error semántico: Función "{nodo.nombre}" no está declarada') from e
        
        if len(nodo.argumentos) != len(info_funcion['parametros']):
            raise Exception(
                f'Error semántico: La función "{nodo.nombre}" espera '
                f'{len(info_funcion["parametros"])} argumentos, '
                f'pero se proporcionaron {len(nodo.argumentos)}'
            )
        
        for i, (arg, (tipo_param, _)) in enumerate(zip(nodo.argumentos, info_funcion['parametros'])):
            tipo_arg = self.analizar(arg)
            if tipo_arg != tipo_param:
                raise Exception(
                    f'Error semántico: El argumento {i+1} de "{nodo.nombre}" '
                    f'es de tipo {tipo_arg}, pero se esperaba {tipo_param}'
                )
        
        return info_funcion['tipo_retorno']
    
    def analizar_NodoIf(self, nodo):
        tipo_condicion = self.analizar(nodo.condicion)
        if tipo_condicion != 'int':
            raise Exception(
                f'Error semántico: La condición del if debe ser de tipo int, '
                f'no {tipo_condicion}'
            )
        
        for instruccion in nodo.cuerpo:
            self.analizar(instruccion)
        for instruccion in nodo.cuerpo_else:
            self.analizar(instruccion)
    
    def analizar_NodoWhile(self, nodo):
        tipo_condicion = self.analizar(nodo.condicion)
        if tipo_condicion != 'int':
            raise Exception(
                f'Error semántico: La condición del while debe ser de tipo int, '
                f'no {tipo_condicion}'
            )
        
        for instruccion in nodo.cuerpo:
            self.analizar(instruccion)

In [ ]:
analizador = AnalizadorSemantico()
try:
    analizador.analizar(arbol_ast)
    print("Análisis semántico exitoso")
    print("Tabla de símbolos global:")
    print("Variables:", analizador.tabla_global.variables)
    print("Funciones:", analizador.tabla_global.funciones)
except Exception as e:
    print(f"Error semántico: {e}")

Error semántico: Error semántico: Operación + entre tipos incompatibles: float y int
